In [2]:
%load_ext autoreload
%autoreload 2
from functools import partial
import pandas as pd
import numpy as np
import uproot
import awkward as ak
import awkward_pandas as akpd
import sys
sys.path.append("..")
from hists.dataframe import *

In [3]:
#input_file = '/grid_mnt/vol_home/llr/cms/cuisset/hgcal/testbeam18/clue3d-dev/src/ClusteringAnalysis/CLUE_clusters_single.root'
input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v20/single-file/data/CLUE_clusters.root'
input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v20/cmssw/sim_proton_v7/CLUE_clusters.root'
for array_i in uproot.iterate(input_file + ":clusters", step_size="100MB", library="ak"):
    series = array_i
    break
comp = DataframeComputations(series)

In [ ]:
cython = comp.clusters3D_intervalHoldingFractionOfEnergy(fraction=0.9, engine="cython")
python = comp.clusters3D_intervalHoldingFractionOfEnergy(fraction=0.9, engine="python")
cython.equals(python)

In [4]:
%load_ext Cython

In [10]:
%%cython --annotate
cimport numpy as np
import numpy as np

# Sum values from i to j inclusive
cdef inline float sumValues(float[:] view, int i, int j):
    cdef float res = 0.
    #print(str(i) + " " + str(j))
    while i <= j:
        res += view[i]
        i += 1
    return res

def computeShortestInterval(np.ndarray series, np.ndarray layers, float fraction) -> tuple[int, int]:
    """ Compute the actual shortest interval. Params : 
        - series : a numpy array, from pandas Series to_numpy (with energy per layer)
        - layers : a numpy array, from pandas Series index to_numpy : layer numbers matching series values (same length as series) """
    cdef float totalEnergyFraction = fraction*np.sum(series)
    cdef int seriesLength = len(series)

    cdef float [:] series_view = series
    cdef long [:] layers_view = layers

    cdef int bestInterval_min = layers_view[0]
    cdef int bestInterval_max = layers_view[-1]
    cdef int j = 0, i_layer, j_layer
    
    for i in range(seriesLength):
        #print("i="+str(i))
        i_layer = layers_view[i]
        if j < i:
            j = i
        
        while sumValues(series_view, i, j) < totalEnergyFraction:
            j += 1
            if j >= seriesLength: # Impossible to find a covering interval at this stage
                return (bestInterval_min, bestInterval_max)
            
        j_layer = layers_view[j]
        if j_layer-i_layer < bestInterval_max - bestInterval_min:
            bestInterval_min = i_layer
            bestInterval_max = j_layer
    
    return (bestInterval_min, bestInterval_max)

In file included from /grid_mnt/vol_home/llr/cms/cuisset/miniconda3/envs/clustering-analysis/lib/python3.11/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940:0,
                 from /grid_mnt/vol_home/llr/cms/cuisset/miniconda3/envs/clustering-analysis/lib/python3.11/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /grid_mnt/vol_home/llr/cms/cuisset/miniconda3/envs/clustering-analysis/lib/python3.11/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /grid_mnt/vol_home/llr/cms/cuisset/.cache/ipython/cython/_cython_magic_55b918ec6b2fec6611efb7faf0bcc829.c:771:
/grid_mnt/vol_home/llr/cms/cuisset/miniconda3/envs/clustering-analysis/lib/python3.11/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it with " \
  ^


In [11]:
grouped = comp.clusters3D_energyClusteredPerLayer.clus2D_energy_sum.groupby(["event", "clus3D_id"])

In [12]:
series = grouped.get_group((18, 1))
computeShortestInterval(series.to_numpy(), series.index.levels[2].to_numpy(), 0.9)

(1, 1)

In [13]:
grouped.agg(lambda series : computeShortestInterval(series.to_numpy(), series.index.levels[2].to_numpy(), 0.9))

event  clus3D_id
0      0            (3, 16)
1      0            (1, 14)
2      0            (4, 19)
3      0            (2, 14)
4      0            (2, 14)
                     ...   
11981  0             (3, 8)
11982  0            (2, 15)
11983  0             (1, 8)
       1            (2, 18)
11984  0            (4, 16)
Name: clus2D_energy_sum, Length: 14812, dtype: object

In [ ]:
series

In [ ]:
computeShortestInterval(np.array([0.1, 1, 5, 0.1], dtype="float32"), np.array([1, 2, 3, 4]), 0.9)

a


(2, 3)

In [ ]:
grouped.get_group((18, 0)).index.levels[2].to_numpy().flags
grouped.get_group((0, 0)).to_numpy().dtype

dtype('float32')

In [ ]:
np.array([0.1, 1, 5, 0.1], dtype="float32").dtype

dtype('float32')